## Relatório final para a publicação de Mineração de Garimpo de Ouro no Brasil

O intuito desta análise é entender o garimpo de ouro no Brasil. Para isso foi usado a base de dados da ANM (disponivel em:####).
O Projeto baseou-se muito em Exploratory Data Analysis (E.D.A).
Norteou a análise entender quantos processos por Ano dão entrada na ANM (na base esse é o evento 333). Quantos desses processos são autorizados a serem explorados (Evento 513). Quanto de área foi pedida para exploração e quanto de área foi autorizada a ser explroado.
Por fim, analisar todo esse contexto, geograficamente, para saber em quais regiões e estados do Brasil acontecem maiores pedidos, em número e em área.

# Importando pacotes do python e lendo as bases da ANM

In [1]:
#Lendo os dataframes
import pandas as pd
import numpy as np
import seaborn as sns
import  matplotlib.pyplot as plt
%matplotlib inline
path = 'C:\\Users\\User\\Dropbox (CPI)\\EduardoMinsky\\Dados_Mineracao\\Dados Tabulares\\Dados_Relacionados\\'

mun = pd.read_csv(path+'ProcessoMunicipio.csv')
substancia = pd.read_csv(path+'ProcessoSubstancia.csv')
eventos = pd.read_csv(path+'ProcessoEvento.csv')
processos = pd.read_csv(path+'Processo.csv')
pessoa = pd.read_csv(path+'ProcessoPessoa.csv')

# Criando dataframes para os diferentes requisitos que queremos

Primeiro vamos preparar nossos dataframes!

In [5]:
#Filtrando os processo ativos!
processos_ativos = processos.query('BTAtivo=="S"')

#Vamos Agora filtrar os processos para Ouro!
ouro = substancia.query('Substancia.str.contains("OURO")',engine = 'python')
#Pegando processos Ativos e com Ouro!
processos_ativos_ouro = processos_ativos[processos_ativos['DSProcesso'].isin(ouro.DSProcesso)]

#Dando Merge ndos DFs de processos ativos de ouro com o DF de Eventos!
processos_ativos_ouro_eventos = processos_ativos_ouro.merge(eventos,how='inner',on='DSProcesso')
processos_ativos_ouro_eventos.columns
columns_keep = ['DSProcesso','BTAtivo','QTAreaHA','Fase_Processo','IDEvento','DTEvento','Evento_']
processos_ativos_ouro_eventos = processos_ativos_ouro_eventos[columns_keep]



    Agora vamos criar um DF com os processos que estão no 333 MAS NAO ESTAO EM MAIS NENHUM OUTRO PROCESSO(isto é, sao processos que estão "parados")
    Outro df de processos que estao no evento 333 AND algum outro evento QUE NAO É O 513 (isto é, sao processos que começaram a andar mas nao estao com a lavra publicada)
    Por fim, processos que chegaram ate o 513, isto é, que estão com a lavra publicada

In [37]:
#Queremos 3 grupos de eventos: Processos somente com evento 333 (processos que não iniciaram a "sua vida")
#Processos com 333 e outros eventos QUE N SAO 513
#E processos que são 333 e chegaram ao 513

#vamos começar fazendo uma nova variavel q reclassifica todos os processos q n sao 333 ou 513 em um numero comum!
processos_ativos_ouro_eventos['DummyEvento'] = np.select(
[
    (processos_ativos_ouro_eventos['IDEvento']!=333) & ( processos_ativos_ouro_eventos['IDEvento']!=513),
    processos_ativos_ouro_eventos['IDEvento']==333,
    processos_ativos_ouro_eventos['IDEvento']==513
    
],
    [
        0,
        333,
        513
        
    ],
    default='none'
)
processos_ativos_ouro_eventos['DummyEvento']=processos_ativos_ouro_eventos['DummyEvento'].astype(int)


#Agora, vamos começar o processo de filtragem para os grupos de eventos que queremos! Primeiro vamos começar filtrando apenas 33
#processo na qual dummyevento é 333 AND processo dummyevento nao é 0 OU  513
ev_333 = processos_ativos_ouro_eventos.query('DummyEvento==333')
ev_333_only = ev_333[~ev_333['DSProcesso'].isin(processos_ativos_ouro_eventos.query('DummyEvento==0 or DummyEvento == 513').DSProcesso)]
ev_333_only['Target']='Evento 333 only' #Está Certo!!!!

#Criando para o evento 513
ev_513 = processos_ativos_ouro_eventos[processos_ativos_ouro_eventos['IDEvento']==513]
ev_513['Target']='Evento 513'

#Agora, vamos pegar processos que são 333 E ZERO na dummy
#Criando dataframe com dummyevento 333 ou dummyevento 0
ev_333_0 = processos_ativos_ouro_eventos[~processos_ativos_ouro_eventos['DSProcesso'].isin(ev_333_only['DSProcesso'])] #Processos em que tem evento 333 e mais alguma coisa
ev_333_0_intermediario = ev_333_0.query('DummyEvento==0')
ev_333_0_only = ev_333_0_intermediario[ev_333_0_intermediario['DSProcesso'].isin(processos_ativos_ouro_eventos.query('DummyEvento==333').DSProcesso)]
ev_333_0_only=ev_333_0_only[~ev_333_0_only['DSProcesso'].isin(ev_513.DSProcesso)]
ev_333_0_only['Target']='Evento 333 and someting else'


C:\Users\User\AppData\Local\Temp\ipykernel_18740\4159502080.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev_333_only['Target']='Evento 333 only' #Está Certo!!!!
C:\Users\User\AppData\Local\Temp\ipykernel_18740\4159502080.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev_513['Target']='Evento 513'


,DSProcesso,BTAtivo,QTAreaHA,Fase_Processo,IDEvento,DTEvento,Evento_,DummyEvento,Target
780,815.187/2022,S,"0,84",Requerimento de Lavra Garimpeira,344,2022-11-09,REQ PLG/EXIGÊNCIA PUBLICADA,0,Evento 333 and someting else
781,815.187/2022,S,"0,84",Requerimento de Lavra Garimpeira,345,2022-12-19,REQ PLG/PRORROGAÇÃO PRAZO EXIGÊNCIA SOLICITADO,0,Evento 333 and someting else
783,815.188/2022,S,"5,04",Requerimento de Lavra Garimpeira,344,2022-11-25,REQ PLG/EXIGÊNCIA PUBLICADA,0,Evento 333 and someting else
784,815.188/2022,S,"5,04",Requerimento de Lavra Garimpeira,345,2023-01-23,REQ PLG/PRORROGAÇÃO PRAZO EXIGÊNCIA SOLICITADO,0,Evento 333 and someting else
786,815.189/2022,S,"9,14",Requerimento de Lavra Garimpeira,344,2022-12-01,REQ PLG/EXIGÊNCIA PUBLICADA,0,Evento 333 and someting else
...,...,...,...,...,...,...,...,...,...
403800,810.365/2011,S,"49,83",Requerimento de Lavra Garimpeira,348,2022-09-01,REQ PLG/CUMPRIMENTO EXIGÊNCIA PROTOCOL,0,Evento 333 and someting else
403801,810.365/2011,S,"49,83",Requerimento de Lavra Garimpeira,573,2022-08-29,REQ PLG/OPÇÃO REGIME AUTORIZAÇÃO PESQ PROTOCOL...,0,Evento 333 and someting else
403802,810.365/2011,S,"49,83",Requerimento de Lavra Garimpeira,573,2022-08-29,REQ PLG/OPÇÃO REGIME AUTORIZAÇÃO PESQ PROTOCOL...,0,Evento 333 and someting else
403803,810.365/2011,S,"49,83",Requerimento de Lavra Garimpeira,793,2013-07-31,REQ PLG/DOCUMENTO DIVERSO PROTOCOLIZADO,0,Evento 333 and someting else


# Filtrando e dando merges

Com este dataframe em mãos, vamos primeiro, filtrar os processos que estão ativos.
  - Para isso, vamos dar um query no df de processos
  - Depois, um inner join nesse df de processos já filtrado com a nossa base criada na célula anterior
  - Por fim, vamos filtrar a base de substancia para ouro e dar um filter com essa base anterior!

In [42]:
processos_ativos_513clean_ouro = ev_513
processos_ativos_333_0_clean_ouro = ev_333_0_only
processos_ativos_333_clean_ouro = ev_333_only
#Retendo apenas processos unicos!
#Antes, vamos transformar as colunas de data em Date!
processos_ativos_513clean_ouro['Data_date'] =  pd.to_datetime(processos_ativos_513clean_ouro.DTEvento)
processos_ativos_333_0_clean_ouro['Data_date'] =  pd.to_datetime(processos_ativos_333_0_clean_ouro.DTEvento)
processos_ativos_333_clean_ouro['Data_date'] =  pd.to_datetime(processos_ativos_333_clean_ouro.DTEvento)

#Mantendo valores unicos (e com ultima data de evento) para o 513!

b_513 = processos_ativos_513clean_ouro[processos_ativos_513clean_ouro['DSProcesso'].duplicated()]
b_513.drop_duplicates('DSProcesso',keep='first',inplace=True)
lista_b_513 = list(b_513['DSProcesso'])
df_unique_513=[]
for i in range(len(lista_b_513)):
    df_unique_513.append(processos_ativos_513clean_ouro[processos_ativos_513clean_ouro['DSProcesso']==lista_b_513[i]])
df_unique_513sorted = []
for i in range(len(df_unique_513)):
    x_513 = df_unique_513[i].sort_values('Data_date',ascending=False)
    y_513=x_513.drop_duplicates('DSProcesso',keep = 'first')
    df_unique_513sorted.append(y_513)
df_unique_sorted_concat_513 = pd.concat(df_unique_513sorted)
processos_ativos_513clean_ouro = processos_ativos_513clean_ouro[~processos_ativos_513clean_ouro['DSProcesso'].isin(df_unique_sorted_concat_513['DSProcesso'])]
processos_ativos_513clean_ouro_clean = pd.concat([df_unique_sorted_concat_513,processos_ativos_513clean_ouro])


#Mantendo valores unicos (e com ultima data de evento) para o processos_ativos_333_0_clean_ouro!
b_333_0 = processos_ativos_333_0_clean_ouro[processos_ativos_333_0_clean_ouro['DSProcesso'].duplicated()]
b_333_0.drop_duplicates('DSProcesso',keep='first',inplace=True)
lista_b_333_0 = list(b_333_0['DSProcesso'])
df_unique_333_0=[]
for i in range(len(lista_b_333_0)):
    df_unique_333_0.append(processos_ativos_333_0_clean_ouro[processos_ativos_333_0_clean_ouro['DSProcesso']==lista_b_333_0[i]])
df_unique_sorted_333_0 = []
for i in range(len(df_unique_333_0)):
    x_333_0 = df_unique_333_0[i].sort_values('Data_date',ascending=False)
    y_333_0=x_333_0.drop_duplicates('DSProcesso',keep = 'first')
    df_unique_sorted_333_0.append(y_333_0)
df_unique_sorted_concat_333_0 = pd.concat(df_unique_sorted_333_0)
processos_ativos_333_0_clean_ouro = processos_ativos_333_0_clean_ouro[~processos_ativos_333_0_clean_ouro['DSProcesso'].isin(df_unique_sorted_concat_333_0['DSProcesso'])]
processos_ativos_333_0_clean_ouro_clean = pd.concat([df_unique_sorted_concat_333_0,processos_ativos_333_0_clean_ouro])
df_allEvents_ouro_ativo_unique = pd.concat([processos_ativos_333_0_clean_ouro_clean,processos_ativos_513clean_ouro_clean,processos_ativos_333_clean_ouro])
df_allEvents_ouro_ativo_unique


C:\Users\User\AppData\Local\Temp\ipykernel_18740\3114913983.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processos_ativos_513clean_ouro['Data_date'] =  pd.to_datetime(processos_ativos_513clean_ouro.DTEvento)
C:\Users\User\AppData\Local\Temp\ipykernel_18740\3114913983.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processos_ativos_333_clean_ouro['Data_date'] =  pd.to_datetime(processos_ativos_333_clean_ouro.DTEvento)
C:\Users\User\AppData\Local\Temp\ipykernel_18740\3114913983.py:13: SettingWithCo

,DSProcesso,BTAtivo,QTAreaHA,Fase_Processo,IDEvento,DTEvento,Evento_,DummyEvento,Target,Data_date
781,815.187/2022,S,"0,84",Requerimento de Lavra Garimpeira,345,2022-12-19,REQ PLG/PRORROGAÇÃO PRAZO EXIGÊNCIA SOLICITADO,0,Evento 333 and someting else,2022-12-19
784,815.188/2022,S,"5,04",Requerimento de Lavra Garimpeira,345,2023-01-23,REQ PLG/PRORROGAÇÃO PRAZO EXIGÊNCIA SOLICITADO,0,Evento 333 and someting else,2023-01-23
787,815.189/2022,S,"9,14",Requerimento de Lavra Garimpeira,345,2023-01-23,REQ PLG/PRORROGAÇÃO PRAZO EXIGÊNCIA SOLICITADO,0,Evento 333 and someting else,2023-01-23
4165,820.434/2007,S,"28,88",Requerimento de Lavra Garimpeira,2,2007-12-20,DOCUMENTO DIVERSO PROTOCOLIZADO,0,Evento 333 and someting else,2007-12-20
4616,820.657/2006,S,"49,70",Requerimento de Lavra Garimpeira,353,2007-05-21,REQ PLG/PEDIDO RECONSIDERAÇÃO PROTOCOLIZADO,0,Evento 333 and someting else,2007-05-21
...,...,...,...,...,...,...,...,...,...,...
395654,803.087/2023,S,"49,41",Requerimento de Lavra Garimpeira,333,2023-04-18,REQ PLG/REQUERIMENTO LAVRA GARIMPEIRA PROTOCOL...,333,Evento 333 only,2023-04-18
398512,806.009/2022,S,"4289,11",Requerimento de Lavra Garimpeira,333,2022-01-21,REQ PLG/REQUERIMENTO LAVRA GARIMPEIRA PROTOCOL...,333,Evento 333 only,2022-01-21
399572,806.094/2020,S,"9863,05",Requerimento de Lavra Garimpeira,333,2020-12-25,REQ PLG/REQUERIMENTO LAVRA GARIMPEIRA PROTOCOL...,333,Evento 333 only,2020-12-25
399574,806.095/2020,S,"9954,97",Requerimento de Lavra Garimpeira,333,2020-12-25,REQ PLG/REQUERIMENTO LAVRA GARIMPEIRA PROTOCOL...,333,Evento 333 only,2020-12-25


Agora, vamos unir esse DF ao dataframe de pessoas:
  - Da seguinte maneira( se o Processo tiver um registro de PF mas tbm de PJ, consideraremos PJ, caso so tenha PF é PF

In [78]:
pessoa_fisica = pessoa.query('Pessoa=="F"')
pessoa_juridica = pessoa.query('Pessoa=="J"')
p_fisica_only = pessoa_fisica[~pessoa_fisica['DSProcesso'].isin(pessoa_juridica['DSProcesso'])]
juridico_certo = pessoa[ ~ pessoa['DSProcesso'].isin(p_fisica_only['DSProcesso'])]
#Criando dummys
p_fisica_only['Pessoa_Dummy'] = 'Pessoa Fisica'
juridico_certo['Pessoa_Dummy'] = 'Pessoa Juridica'
p_fisica_only.drop_duplicates('DSProcesso',keep = 'first',inplace=True)
juridico_certo.drop_duplicates('DSProcesso',keep = 'first',inplace=True)
pessoas_certo = pd.concat([p_fisica_only,juridico_certo])


C:\Users\User\AppData\Local\Temp\ipykernel_18740\1617136912.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_fisica_only['Pessoa_Dummy'] = 'Pessoa Fisica'
C:\Users\User\AppData\Local\Temp\ipykernel_18740\1617136912.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  juridico_certo['Pessoa_Dummy'] = 'Pessoa Juridica'
C:\Users\User\AppData\Local\Temp\ipykernel_18740\1617136912.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the document

,Unnamed: 0,DSProcesso,IDPessoa,IDTipoRelacao,IDTipoResponsabilidadeTecnica,IDTipoRepresentacaoLegal,DTPrazoArrendamento,DTInicioVigencia,DTFimVigencia,Pessoa,Pessoa_NOME,TipoRelacao,Responsa_tecnica,Representacao_Legal,Pessoa_Dummy


Por fim, vamos pegar os processos por Estado e municipio! Vamos tirar processos que ocorrem em mais de um municipio!

In [76]:
mun.drop_duplicates('DSProcesso',keep = 'first',inplace = True)

# Criando dataframe Final!

In [94]:
#Merge de pessoas com nosso df!
df_allEvents_ouro_ativo_unique_pessoa  = df_allEvents_ouro_ativo_unique.merge(pessoas_certo,how='inner',on='DSProcesso')
columns_keep_2 = ['DSProcesso','BTAtivo','QTAreaHA','Fase_Processo','IDEvento','Evento_','DummyEvento','Target','Data_date','Pessoa_NOME','Pessoa_Dummy']
df_allEvents_ouro_ativo_unique_pessoa = df_allEvents_ouro_ativo_unique_pessoa[columns_keep_2]
#Merge deste ultimo df com df de municipio
df_allEvents_ouro_ativo_unique_pessoa_municipio = df_allEvents_ouro_ativo_unique_pessoa.merge(mun,on='DSProcesso',how='inner')
del df_allEvents_ouro_ativo_unique_pessoa_municipio['Unnamed: 0']

#Agrupando os dados de Estado por Regiao
#Agrupando os estados por Regiao (Norte, sul, Sudeste,CentroOeste, Nordeste)
sul =['RS','SC','PR']
sudeste = ['SP','MG','RJ','ES']
centrooeste =['MS','MT','GO','DF']
nordeste = ['BA','PI','MA','CE','RN','PB','PE','AL','SE']
norte = ['AC','RO','AM','RR','PA','TO','AP']
df_allEvents_ouro_ativo_unique_pessoa_municipio['Regiao'] = ['Sul' if x in sul else 'Sudeste' if x in sudeste else 'Centro_Oeste' if x in centrooeste else 'Nordeste' if x in nordeste else 'Norte' if x in norte else 'SemEstado' for x in df_allEvents_ouro_ativo_unique_pessoa_municipio['UF']]
#Salvando!!
df_allEvents_ouro_ativo_unique_pessoa_municipio.to_csv('DataFrameFinalMineracaoOuro.csv')


,DSProcesso,BTAtivo,QTAreaHA,Fase_Processo,IDEvento,Evento_,DummyEvento,Target,Data_date,Pessoa_NOME,Pessoa_Dummy,IDMunicipio,Municipios,UF,Regiao
0,815.187/2022,S,"0,84",Requerimento de Lavra Garimpeira,345,REQ PLG/PRORROGAÇÃO PRAZO EXIGÊNCIA SOLICITADO,0,Evento 333 and someting else,2022-12-19,Diego Fortunato Comin,Pessoa Fisica,4202909,BRUSQUE,SC,Sul
1,815.188/2022,S,"5,04",Requerimento de Lavra Garimpeira,345,REQ PLG/PRORROGAÇÃO PRAZO EXIGÊNCIA SOLICITADO,0,Evento 333 and someting else,2023-01-23,Diego Fortunato Comin,Pessoa Fisica,4202909,BRUSQUE,SC,Sul
2,815.189/2022,S,"9,14",Requerimento de Lavra Garimpeira,345,REQ PLG/PRORROGAÇÃO PRAZO EXIGÊNCIA SOLICITADO,0,Evento 333 and someting else,2023-01-23,Diego Fortunato Comin,Pessoa Fisica,4202909,BRUSQUE,SC,Sul
3,820.434/2007,S,"28,88",Requerimento de Lavra Garimpeira,2,DOCUMENTO DIVERSO PROTOCOLIZADO,0,Evento 333 and someting else,2007-12-20,ADEMIR LIGUORI,Pessoa Fisica,3523305,ITARIRI,SP,Sudeste
4,820.657/2006,S,"49,70",Requerimento de Lavra Garimpeira,353,REQ PLG/PEDIDO RECONSIDERAÇÃO PROTOCOLIZADO,0,Evento 333 and someting else,2007-05-21,Edson Sansone,Pessoa Fisica,3523305,ITARIRI,SP,Sudeste
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17839,803.087/2023,S,"49,41",Requerimento de Lavra Garimpeira,333,REQ PLG/REQUERIMENTO LAVRA GARIMPEIRA PROTOCOL...,333,Evento 333 only,2023-04-18,Felipe Augusto Siqueira Pinheiro,Pessoa Fisica,2202109,CAMPINAS DO PIAUÍ,PI,Nordeste
17840,806.009/2022,S,"4289,11",Requerimento de Lavra Garimpeira,333,REQ PLG/REQUERIMENTO LAVRA GARIMPEIRA PROTOCOL...,333,Evento 333 only,2022-01-21,CLEBER DOS SANTOS ALVES,Pessoa Juridica,2103158,CENTRO DO GUILHERME,MA,Nordeste
17841,806.094/2020,S,"9863,05",Requerimento de Lavra Garimpeira,333,REQ PLG/REQUERIMENTO LAVRA GARIMPEIRA PROTOCOL...,333,Evento 333 only,2020-12-25,CLEBER DOS SANTOS ALVES,Pessoa Juridica,2102606,CÂNDIDO MENDES,MA,Nordeste
17842,806.095/2020,S,"9954,97",Requerimento de Lavra Garimpeira,333,REQ PLG/REQUERIMENTO LAVRA GARIMPEIRA PROTOCOL...,333,Evento 333 only,2020-12-25,CLEBER DOS SANTOS ALVES,Pessoa Juridica,2100550,AMAPÁ DO MARANHÃO,MA,Nordeste


# Fazendo as análises exploratória dos dados!